In [1]:
# Gerekli kütüphanelerin tanımlanması ve ortam ayarları
import os
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib  # Model kaydetme/yükleme için

# Scikit-learn modülleri
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Uyarıları filtrele
warnings.filterwarnings('ignore')

# Windows işlemci çekirdek hatası için düzeltme (Joblib/Loky)
os.environ["LOKY_MAX_CPU_COUNT"] = "4"

print("Kütüphaneler yüklendi ve ortam hazır.")

Kütüphaneler yüklendi ve ortam hazır.


In [2]:
# Veri setinin yüklenmesi
try:
    df = pd.read_csv('owid-energy-data.csv')
except FileNotFoundError:
    print("Hata: Veri dosyası bulunamadı. Lütfen dizini kontrol edin.")

# 2022 yılı verilerinin filtrelenmesi ve bölge (World/Europe vb.) temizliği
df_2022 = df[df['year'] == 2022].copy()
df_clean = df_2022.dropna(subset=['iso_code'])
df_clean = df_clean[~df_clean['iso_code'].str.contains('OWID')]

# Analiz için kullanılacak özellik havuzu
features = [
    'biofuel_share_elec', 'coal_share_elec', 'solar_share_elec',
    'hydro_share_elec', 'nuclear_share_elec', 'oil_share_elec',
    'gas_share_elec', 'wind_share_elec', 'carbon_intensity_elec',
    'gdp', 'population', 'energy_per_capita'
]

# İlgili sütunların seçilmesi
data = df_clean[features].copy()

# Eksik verilerin medyan ile doldurulması (Imputation)
for col in data.columns:
    medyan = data[col].median()
    data[col].fillna(medyan, inplace=True)

# Verinin ölçeklendirilmesi (Standardizasyon)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data)
X_df = pd.DataFrame(X_scaled, columns=data.columns)

print(f"Veri ön işleme tamamlandı. Analiz edilecek ülke sayısı: {len(X_df)}")

Veri ön işleme tamamlandı. Analiz edilecek ülke sayısı: 220


In [3]:
# --- Yöntem A: İstatistiksel Özellik Seçimi (Korelasyon Analizi) ---

# Korelasyon matrisinin hesaplanması
corr_matrix = X_df.corr().abs()

# Üst üçgen matrisin seçilmesi
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# %90 üzeri korelasyona sahip sütunların tespiti ve düşürülmesi
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
X_final = X_df.drop(columns=to_drop)

print("Yüksek korelasyonlu değişkenler elendi:", to_drop)
print(f"Model için kalan özellik sayısı: {X_final.shape[1]}")

Yüksek korelasyonlu değişkenler elendi: []
Model için kalan özellik sayısı: 12


In [4]:
# --- Base Modellerin Kurulumu ve Eğitimi ---

# Küme sayısı (k) parametresi (Başlangıç için 3 seçildi)
k = 3

# 1. Model: K-Means
kmeans_model = KMeans(n_clusters=k, random_state=42, n_init=10)
kmeans_labels = kmeans_model.fit_predict(X_final)

# 2. Model: Agglomerative Hierarchical Clustering
agg_model = AgglomerativeClustering(n_clusters=k)
agg_labels = agg_model.fit_predict(X_final)

# Başarı Metriklerinin Hesaplanması
results = {
    "K-Means Silhouette": silhouette_score(X_final, kmeans_labels),
    "K-Means Davies-Bouldin": davies_bouldin_score(X_final, kmeans_labels),
    "Hierarchical Silhouette": silhouette_score(X_final, agg_labels),
    "Hierarchical Davies-Bouldin": davies_bouldin_score(X_final, agg_labels)
}

print("Modeller başarıyla eğitildi.\n")
print("-" * 30)
print(f"K-Means Silhouette Skoru: {results['K-Means Silhouette']:.4f}")
print(f"Hiyerarşik Silhouette Skoru: {results['Hierarchical Silhouette']:.4f}")
print("-" * 30)

Modeller başarıyla eğitildi.

------------------------------
K-Means Silhouette Skoru: 0.2245
Hiyerarşik Silhouette Skoru: 0.1973
------------------------------


In [5]:
# --- Modellerin Diske Kaydedilmesi ---

# Klasör kontrolü
if not os.path.exists('saved_models'):
    os.makedirs('saved_models')

# Modellerin ve Scaler nesnesinin kaydedilmesi
joblib.dump(kmeans_model, 'saved_models/kmeans_model_mukhammad.pkl')
joblib.dump(agg_model, 'saved_models/agg_model_mukhammad.pkl')
joblib.dump(scaler, 'saved_models/scaler_mukhammad.pkl')
# Seçilen özellikleri de kaydetmek pratiktir (hangi sütunların kaldığını bilmek için)
joblib.dump(X_final.columns.tolist(), 'saved_models/selected_features.pkl')

print("Modeller 'saved_models' klasörüne kaydedildi.")

# --- Kontrol: Kaydedilen Modelin Yüklenmesi ---
print("\n--- Model Yükleme Testi ---")

loaded_kmeans = joblib.load('saved_models/kmeans_model_mukhammad.pkl')
loaded_features = joblib.load('saved_models/selected_features.pkl')

print("Model başarıyla yüklendi.")
print(f"Yüklenen modelin küme merkezleri boyutu: {loaded_kmeans.cluster_centers_.shape}")
print(f"Modelin kullandığı özellikler: {loaded_features}")

Modeller 'saved_models' klasörüne kaydedildi.

--- Model Yükleme Testi ---
Model başarıyla yüklendi.
Yüklenen modelin küme merkezleri boyutu: (3, 12)
Modelin kullandığı özellikler: ['biofuel_share_elec', 'coal_share_elec', 'solar_share_elec', 'hydro_share_elec', 'nuclear_share_elec', 'oil_share_elec', 'gas_share_elec', 'wind_share_elec', 'carbon_intensity_elec', 'gdp', 'population', 'energy_per_capita']


In [6]:

# Bu kodu çalıştırarak tablonun ilk satırını (Kontrol) doldurabilirsiniz.
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Veriyi Hazırla
df = pd.read_csv('owid-energy-data.csv')
df = df[(df['year'] == 2022) & (~df['iso_code'].str.contains('OWID', na=True))]
features = [
    'coal_share_elec', 'gas_share_elec', 'hydro_share_elec', 
    'nuclear_share_elec', 'oil_share_elec', 'solar_share_elec', 
    'wind_share_elec', 'biofuel_share_elec', 'carbon_intensity_elec'
]
data = df[features].copy()

# Temizle ve Ölçeklendir
data_imputed = SimpleImputer(strategy='median').fit_transform(data)
data_scaled = StandardScaler().fit_transform(data_imputed)

# Modeli Çalıştır (K-Means, k=3)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10).fit(data_scaled)
sil = silhouette_score(data_scaled, kmeans.labels_)
db = davies_bouldin_score(data_scaled, kmeans.labels_)

print(f"Kontrol (Silhouette): {sil:.6f}")
print(f"Kontrol (Davies-Bouldin): {db:.6f}")

TABLOYA YAZILACAK DEĞERLER:
Kontrol (Silhouette): 0.260851
Kontrol (Davies-Bouldin): 1.563520


In [8]:
# HÜCRE 7: Modellerin Diske Kaydedilmesi (Düzeltilmiş)
import joblib
import os

# Klasör kontrolü: 'saved_models' klasörü yoksa oluştur
if not os.path.exists('saved_models'):
    os.makedirs('saved_models')

print("Modeller kaydediliyor...")

# 1. K-Means Modelini Kaydet
# Önceki hücrede modelin ismi 'kmeans_model' idi
joblib.dump(kmeans_model, 'saved_models/kmeans_model_mukhammad.pkl')

# 2. Hiyerarşik Modelini Kaydet
# Önceki hücrede modelin ismi 'agg_model' idi
joblib.dump(agg_model, 'saved_models/agg_model_mukhammad.pkl')

# 3. Scaler (Ölçekleyici) Kaydet
# Yeni verileri aynı standarta getirmek için gerekli
joblib.dump(scaler, 'saved_models/scaler_mukhammad.pkl')

# 4. Seçilen Özellik Listesini Kaydet
# Hangi sütunları kullandığımızı hatırlamak için
joblib.dump(X_final.columns.tolist(), 'saved_models/selected_features.pkl')

print("BAŞARILI: Tüm modeller 'saved_models' klasörüne kaydedildi.")
print("Kaydedilen dosyalar:")
print("- kmeans_model_mukhammad.pkl")
print("- agg_model_mukhammad.pkl")
print("- scaler_mukhammad.pkl")
print("- selected_features.pkl")

Modeller kaydediliyor...
BAŞARILI: Tüm modeller 'saved_models' klasörüne kaydedildi.
Kaydedilen dosyalar:
- kmeans_model_mukhammad.pkl
- agg_model_mukhammad.pkl
- scaler_mukhammad.pkl
- selected_features.pkl
